In [1]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" /kaggle/input/nvidia-apex/apex-880ab92

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-kmf8rmp8
Created temporary directory: /tmp/pip-req-tracker-bmyydhxe
Created requirements tracker '/tmp/pip-req-tracker-bmyydhxe'
Created temporary directory: /tmp/pip-install-axhnxcy0
Processing /kaggle/input/nvidia-apex/apex-880ab92
  Created temporary directory: /tmp/pip-req-build-o15qdqbv
  Added file:///kaggle/input/nvidia-apex/apex-880ab92 to build tracker '/tmp/pip-req-tracker-bmyydhxe'
    Running setup.py (path:/tmp/pip-req-build-o15qdqbv/setup.py) egg_info for package from file:///kaggle/input/nvidia-apex/apex-880ab92
    Running command python setup.py egg_info
    torch.__version__  =  1.2.0
    running egg_info
    creating pip-egg-info/apex.egg-info
    writing pip-egg-info/apex.e

In [2]:
from __future__ import absolute_import, division, print_function

import os, sys, re, gc, pickle, operator, shutil, copy
import time, datetime

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, Sampler

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from apex import amp
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertConfig
from pytorch_pretrained_bert.modeling import BertModel, BertPreTrainedModel
device = torch.device('cuda')

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
WORK_DIR = "./"
DATA_DIR = "/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/"
BERT_MODEL_PATH = '/kaggle/input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
output_model_file = "bert_models.pt"

SEED = 2019

MAX_SEQUENCE_LENGTH = 220
epochs_for_sched = 1
checkpoint_iter = 20000
lr = 2.5e-5
batch_size = 16
warmup_proportion = 0.05
grad_accumulation_steps = 2

identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

aux_columns = ['severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']

label_column = 'target'
pred_column = 'prediction'
text_column = 'comment_text'

In [5]:
# Seed for randomness in pytorch
def seed_torch(seed=SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
# Convert target and identity columns to booleans
def convert_dataframe_to_bool(df):
    def convert_to_bool(df, col_name):
        df[col_name] = np.where(df[col_name] >= 0.5, True, False)

    bool_df = df.copy()
    for col in [label_column] + identity_columns + aux_columns:
        convert_to_bool(bool_df, col)
    return bool_df

In [6]:
# Tokenizing the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(texts, max_seq_length, tokenizer):
    max_seq_length -= 2
    all_tokens = []

    for text in texts:
        tokens = tokenizer.tokenize(text)
        if len(tokens) > max_seq_length:
            tokens = tokens[-max_seq_length:]
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens+["[SEP]"])
        all_tokens.append(one_token)

    return np.array(all_tokens)

In [7]:
# Prepare dataset and dataloader

class Toxic_comments(Dataset):

    def __init__(self, tokenized_comments, targets=None, split=None, maxlen=256):
        self.comments = tokenized_comments
        self.targets = targets
        self.split = split
        assert self.split in {'train', 'valid', 'test'}
        self.maxlen = maxlen

    def __getitem__(self, index):
        comment = self.comments[index]
        if self.targets is not None:
            target = self.targets[index]
            return comment, torch.FloatTensor(target)
        else:
            return comment

    def __len__(self):
        return len(self.comments)

    def get_lens(self):
        lengths = np.fromiter(
            ((min(self.maxlen, len(seq))) for seq in self.comments),
            dtype=np.int32)
        return lengths

    def collate_fn(self, batch):
        """
        Collate function for sequence bucketing
        Note: this need not be defined in this Class, can be standalone.

        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of comments, and targets
        """

        if self.split in ('train', 'valid'):
            comments, targets = zip(*batch)
        else:
            comments = batch

        lengths = [len(c) for c in comments]
        maxlen = max(lengths)
        padded_comments = []
        for i, c in enumerate(comments):
            padded_comments.append(c+[0]*(maxlen - lengths[i]))

        if self.split in ('train', 'valid'):
            return torch.LongTensor(padded_comments), torch.stack(targets)
        else:
            return torch.LongTensor(padded_comments)


class BucketSampler(Sampler):

    def __init__(self, data_source, sort_lens, bucket_size=None, batch_size=1024, shuffle_data=True):
        super().__init__(data_source)
        self.shuffle = shuffle_data
        self.batch_size = batch_size
        self.sort_lens = sort_lens
        self.bucket_size = bucket_size if bucket_size is not None else len(sort_lens)
        self.weights = None

        if not shuffle_data:
            self.index = self.prepare_buckets()
        else:
            self.index = None

    def set_weights(self, weights):
        assert weights >= 0
        total = np.sum(weights)
        if total != 1:
            weights = weights / total
        self.weights = weights

    def __iter__(self):
        indices = None
        if self.weights is not None:
            total = len(self.sort_lens)
            indices = np.random.choice(total, (total,), p=self.weights)
        if self.shuffle:
            self.index = self.prepare_buckets(indices)
        return iter(self.index)

    def get_reverse_indexes(self):
        indexes = np.zeros((len(self.index),), dtype=np.int32)
        for i, j in enumerate(self.index):
            indexes[j] = i
        return indexes

    def __len__(self):
        return len(self.sort_lens)

    def prepare_buckets(self, indices=None):
        lengths = - self.sort_lens
        assert self.bucket_size % self.batch_size == 0 or self.bucket_size == len(lengths)

        if indices is None:
            if self.shuffle:
                indices = shuffle(np.arange(len(lengths), dtype=np.int32))
                lengths = lengths[indices]
            else:
                indices = np.arange(len(lengths), dtype=np.int32)

        #  bucket iterator
        def divide_chunks(l, n):
            if n == len(l):
                yield np.arange(len(l), dtype=np.int32), l
            else:
                # looping till length l
                for i in range(0, len(l), n):
                    data = l[i:i + n]
                    yield np.arange(i, i + len(data), dtype=np.int32), data

        new_indices = []
        extra_batch_idx = None
        for chunk_index, chunk in divide_chunks(lengths, self.bucket_size):
            # sort indices in bucket by descending order of length
            indices_sorted = chunk_index[np.argsort(chunk)]

            batch_idxes = []
            for _, batch_idx in divide_chunks(indices_sorted, self.batch_size):
                if len(batch_idx) == self.batch_size:
                    batch_idxes.append(batch_idx.tolist())
                else:
                    assert extra_batch_idx is None
                    assert batch_idx is not None
                    extra_batch_idx = batch_idx.tolist()

            # shuffling batches within buckets
            if self.shuffle:
                batch_idxes = shuffle(batch_idxes)
            for batch_idx in batch_idxes:
                new_indices.extend(batch_idx)

        if extra_batch_idx is not None:
            new_indices.extend(extra_batch_idx)

        if not self.shuffle:
            self.original_indices = np.argsort(indices_sorted).tolist()
        return indices[new_indices]


def prepare_loader(x, y=None, batch_size=1024, split=None):
    assert split in {'train', 'valid', 'test'}
    dataset = Toxic_comments(x, y, split, MAX_SEQUENCE_LENGTH)
    if split == 'train':
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                bucket_size=batch_size*50, batch_size=batch_size)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn)
    else:
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                batch_size=batch_size, shuffle_data=False)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn), sampler.original_indices

In [8]:
# Metrics

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, pred_column):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[pred_column])

def compute_bpsn_auc(df, subgroup, label, pred_column):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[pred_column])

def compute_bnsp_auc(df, subgroup, label, pred_column):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[pred_column])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

def calculate_overall_auc(df, pred_col, label_col):
    true_labels = df[label_col]
    predicted_labels = df[pred_col]
    return roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [9]:
# Functions for the training process

class NetSolver(object):

    def __init__(self, model, criterion, optimizer, scheduler, print_freq, filepath):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.print_freq = print_freq
        self.filepath = filepath

        self.model = self.model.to(device)
        self.criterion = self.criterion.to(device)
        self._reset()

    def _reset(self):
        """Set up some book-keeping variables for optimization.
        """
        self.best_val_loss = 1e4
        self.best_val_auc = 0.
        self.loss_history = []
        self.val_loss_history = []
        self.auc_history = []
        self.val_auc_history = []
        self.val_unbias_auc_history = []
        self.val_preds = []
        self.models = {}

    def save_checkpoint(self, iteration):
        """Save model checkpoint.
        """
        self.models[f'ckpt_{iteration}'] = self.model.state_dict()
    
    def save_final_state(self):
        """Save final states.
        """
        state = {'model': self.models,
                 'optimizer': self.optimizer.state_dict()
                 }
        torch.save(state, self.filepath)

    def forward_pass(self, x, y):
        """Forward pass through the network.
        """
        x, y = x.to(device=device, dtype=torch.long), y.to(device=device, dtype=torch.float)
        scores = self.model(x)
        loss = self.criterion(scores, y)
        return loss, torch.sigmoid(scores)

    def train(self, loaders, iterations, val_df, val_original_indices, start_time):
        """Weight of network updated by apex, grad accumulation, model checkpoint.
        """
        train_loader, val_loader = loaders
        loader = iter(train_loader)
        running_loss = 0.
        self.optimizer.zero_grad()

        # start training for iterations
        for i in range(iterations):
            self.model.train()

            try:
                x, y = next(loader)
            except:
                loader = iter(train_loader)
                x, y = next(loader)
            loss, _ = self.forward_pass(x, y)
            
#             loss.backward()
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()

            # gradient accumulation for larger batch size effect with smaller memory usage
            if (i+1) % grad_accumulation_steps == 0:   # Wait for several backward steps
                self.optimizer.step()                  # Now we can do an optimizer step
                self.optimizer.zero_grad()
                self.scheduler.step()

            running_loss += loss.item()
            
            # verbose and checkpoint
            if (i+1) % self.print_freq == 0 or (i+1) == iterations:
                print(f'Iteration {i+1}:')
                train_auc, _ = self.check_auc(train_loader, num_batches=50)
                print('{"metric": "Loss", "value": %.4f}' % (running_loss/(i+1),))
                print('{"metric": "AUC", "value": %.4f}' % (train_auc,))
                
                is_print = False if (i+1) < iterations else True
                val_auc, val_unbias_auc, val_loss, val_scores = self.check_auc(
                    val_loader, df=val_df, idxs=val_original_indices, is_print=is_print)
                print('{"metric": "Val. Loss", "value": %.4f}' % (val_loss,))
                print('{"metric": "Val. AUC", "value": %.4f}' % (val_auc,))
                print('{"metric": "Val. Unbiased AUC", "value": %.4f}' % (val_unbias_auc,))
                
                self.loss_history.append(running_loss/(i+1))
                self.val_loss_history.append(val_loss)
                self.auc_history.append(train_auc)
                self.val_auc_history.append(val_auc)
                self.val_unbias_auc_history.append(val_unbias_auc)
                self.val_preds.append(val_scores)
                
                if val_loss < self.best_val_loss:
                    print('updating best val loss...')
                    self.best_val_loss = val_loss
                if val_unbias_auc > self.best_val_auc:
                    print('updating best val auc...')
                    self.best_val_auc = val_unbias_auc
                    self.save_checkpoint(i+1)
                
                torch.cuda.empty_cache()
                print()
                
            if (time.time() - start_time) > 29000:
                break
        
        self.save_final_state()

    def check_auc(self, loader, num_batches=None, df=None, idxs=None, is_print=False):
        """Calculate metrics for validation
        """
        self.model.eval()
        targets, scores, losses = [], [], []
        with torch.no_grad():
            for t, (x, y) in enumerate(loader):
                l, score = self.forward_pass(x, y)
                targets.append((y[:,0].cpu().numpy()>=0.5).astype(int))
                scores.append(score[:,0].cpu().numpy())
                losses.append(l.item())
                if num_batches is not None and (t+1) == num_batches:
                    break

        targets = np.concatenate(targets)
        scores = np.concatenate(scores)
        auc = roc_auc_score(targets, scores)
        loss = np.mean(losses)

        if df is not None:
            unbias_auc = check_unbias_auc(df, scores[idxs], is_print)
            return auc, unbias_auc, loss, scores[idxs]

        return auc, loss
    

def check_unbias_auc(df, scores, print_table=False):
    """Calculate metrics for validation
    """
    df[pred_column] = scores
    bias_metrics_df = compute_bias_metrics_for_model(df, identity_columns, pred_column, label_column)
    unbias_auc = get_final_metric(bias_metrics_df, calculate_overall_auc(df, pred_column, label_column))
    if print_table:
        print(bias_metrics_df)
    return unbias_auc

In [10]:
class OneCycleScheduler(object):
    # one-cycle scheduler
    SCHEDULES = set(['cosine', 'linear', 'linear_cosine'])

    def __init__(self, optimizer, iterations, sched_profile='cosine', max_lr=3e-3,
                 moms=(.95, .85), div_factor=25, warmup=0.3, final_div=None):

        self.optimizer = optimizer
        assert sched_profile in self.SCHEDULES
        self.sched_profile = sched_profile

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
            self.init_lrs = [lr/div_factor for lr in self.max_lrs]
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)
            self.init_lrs = [max_lr/div_factor] * len(optimizer.param_groups)

        self.final_div = final_div
        if self.final_div is None: self.final_div = div_factor*1e4
        self.final_lrs = [lr/self.final_div for lr in self.max_lrs]
        self.moms = moms

        self.total_iteration = iterations
        self.up_iteration = int(self.total_iteration * warmup)
        self.down_iteration = self.total_iteration - self.up_iteration

        self.curr_iter = 0
        self._assign_lr_mom(self.init_lrs, [moms[0]]*len(optimizer.param_groups))

    def _assign_lr_mom(self, lrs, moms):
        for param_group, lr, mom in zip(self.optimizer.param_groups, lrs, moms):
            param_group['lr'] = lr
            param_group['betas'] = (mom, 0.999)

    def _annealing_cos(self, start, end, pct):
        cos_out = np.cos(np.pi * pct) + 1
        return end + (start-end)/2 * cos_out

    def _annealing_linear(self, start, end, pct):
        return start + pct * (end-start)
    
    def _annealing_function(self, curr_iter):
        if self.sched_profile == 'cosine':
            return self._annealing_cos
        if self.sched_profile == 'linear':
            return self._annealing_linear
        if self.sched_profile == 'linear_cosine':
            if curr_iter <= self.up_iteration:
                return self._annealing_linear
            else:
                return self._annealing_cos
    
    def step(self):
        self.curr_iter += 1
        anneal = self._annealing_function(self.curr_iter)

        if self.curr_iter <= self.up_iteration:
            pct = self.curr_iter / self.up_iteration
            curr_lrs = [anneal(min_lr, max_lr, pct) \
                            for min_lr, max_lr in zip(self.init_lrs, self.max_lrs)]
            curr_moms = [anneal(self.moms[0], self.moms[1], pct) \
                            for _ in range(len(self.optimizer.param_groups))]
        else:
            pct = (self.curr_iter-self.up_iteration) / self.down_iteration
            curr_lrs = [anneal(max_lr, final_lr, pct) \
                            for max_lr, final_lr in zip(self.max_lrs, self.final_lrs)]
            curr_moms = [anneal(self.moms[1], self.moms[0], pct) \
                            for _ in range(len(self.optimizer.param_groups))]

        self._assign_lr_mom(curr_lrs, curr_moms)

def lr_range_test(train_loader, model, optimizer, criterion, start_lr=1e-7,
                  end_lr=10, num_it=100, stop_div=True):
    epochs = int(np.ceil(num_it/len(train_loader)))
    n_groups = len(optimizer.param_groups)

    if isinstance(start_lr, list) or isinstance(start_lr, tuple):
        if len(start_lr) != n_groups:
            raise ValueError("expected {} max_lr, got {}".format(n_groups, len(start_lr)))
        start_lrs = list(start_lr)
    else:
        start_lrs = [start_lr] * n_groups

    if isinstance(end_lr, list) or isinstance(end_lr, tuple):
        if len(end_lr) != n_groups:
            raise ValueError("expected {} max_lr, got {}".format(n_groups, len(end_lr)))
        end_lrs = list(end_lr)
    else:
        end_lrs = [end_lr] * n_groups

    curr_lrs = start_lrs*1
    for param_group, lr in zip(optimizer.param_groups, curr_lrs):
        param_group['lr'] = lr

    n, lrs_logs, loss_log = 0, [], []

    for e in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device=device, dtype=torch.long), y.to(device=device, dtype=torch.float)
            scores = model(x)
            loss = criterion(scores, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            lrs_logs.append(curr_lrs)
            loss_log.append(loss.item())

            # update best loss
            if n == 0:
                best_loss, n_best = loss.item(), n
            else:
                if loss.item() < best_loss:
                    best_loss, n_best = loss.item(), n

            # update lr per iter with exponential schedule
            n += 1
            curr_lrs = [lr * (end_lr/lr) ** (n/num_it) for lr, end_lr in zip(start_lrs, end_lrs)]
            for param_group, lr in zip(optimizer.param_groups, curr_lrs):
                param_group['lr'] = lr

            # stopping condition
            if n == num_it or (stop_div and (loss.item() > 4*best_loss or torch.isnan(loss))):
                break

    print('minimum loss {}, at lr {}'.format(best_loss, lrs_logs[n_best]))
    return lrs_logs, loss_log

In [11]:
# Classifier
class BertForCustomClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertForCustomClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropouts = nn.ModuleList([nn.Dropout(config.hidden_dropout_prob) for _ in range(5)])
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                h = self.classifier(dropout(pooled_output))
            else:
                h += self.classifier(dropout(pooled_output))
        return h / len(self.dropouts)

# Loss
class UnbiasLoss(nn.Module):
    def __init__(self, main_loss_weight=3.0):
        super(UnbiasLoss, self).__init__()
        self.alpha = main_loss_weight
    def forward(self, pred_scores, labels):
        main_loss = nn.BCEWithLogitsLoss(weight=labels[:,-1])(pred_scores[:,0], labels[:,0])
        aux_loss = nn.BCEWithLogitsLoss()(pred_scores[:,1:], labels[:,1:-1])
        return self.alpha * main_loss + aux_loss

# Build model and optimizer
def model_optimizer_init(ft_lrs, num_labels=16):
    print("Building model and optimizer...")
    pre_model = BertForSequenceClassification.from_pretrained('../working', num_labels=num_labels)
    bert_config = BertConfig(WORK_DIR + 'bert_config.json')
    model = BertForCustomClassification(bert_config, num_labels=num_labels)
    model.bert = copy.deepcopy(pre_model.bert)

    params_bert = list(model.bert.parameters())
    params_cls = list(model.classifier.parameters())

    optimizer_grouped_parameters = [
        {'params': params_bert, 'lr':ft_lrs[0]},
        {'params': params_cls, 'lr':ft_lrs[1]}
        ]
    
    optimizer = torch.optim.Adam(optimizer_grouped_parameters)
    
    return model, optimizer
    

# Translate model from tensorflow to pytorch
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

# Save config files in the same path as pretrained model
# for model reloading (inference, resume training etc.)
shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/L

'./bert_config.json'

In [12]:
def train_val_split(train_x, train_y):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    cv_indices = [(tr_idx, val_idx) for tr_idx, val_idx in kf.split(train_x, train_y)]
    return cv_indices

def load_and_preproc():
    train_df = pd.read_csv(DATA_DIR+'train.csv')
    train_df[identity_columns] = train_df[identity_columns].copy().fillna(0)

    sample_weights = np.ones(len(train_df))
    sample_weights += train_df[identity_columns].values.sum(1) * 3
    sample_weights += train_df[label_column].values * 8
    sample_weights /= sample_weights.max()
    train_tars = train_df[[label_column]+aux_columns+identity_columns].values
    train_tars = np.hstack([train_tars, sample_weights[:,None]]).astype('float32')

    train_df = convert_dataframe_to_bool(train_df)
    df = train_df[[label_column]+identity_columns].copy()
    df[label_column] = df[label_column].astype('uint8')

    return train_df[text_column], train_tars, df

In [13]:
# %%time

# train_seq, train_tars, trn_df = load_and_preproc()
# cv_indices = train_val_split(train_seq, (train_tars[:,0]>=0.5).astype(int))
# trn_idx, val_idx = cv_indices[0]

# print('tokenizing...')
# t0 = time.time()
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, do_lower_case=True)
# train_seq = convert_lines(train_seq[trn_idx[:20000]], MAX_SEQUENCE_LENGTH, tokenizer)
# print('tokenizing complete in {:.0f} seconds.'.format(time.time()-t0))

# x_train, x_val = train_seq[:16000], train_seq[16000:]
# y_train, y_val = train_tars[trn_idx[:16000]], train_tars[trn_idx[16000:20000]]
# train_loader = prepare_loader(x_train, y_train, batch_size, split='train')
# val_loader, val_original_indices = prepare_loader(x_val, y_val, 32, split='valid')
# val_df = trn_df.iloc[trn_idx[16000:20000]]

In [14]:
%%time

train_seq, train_tars, trn_df = load_and_preproc()
cv_indices = train_val_split(train_seq, (train_tars[:,0]>=0.5).astype(int))
trn_idx, val_idx = cv_indices[0]

print('tokenizing...')
t0 = time.time()
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, do_lower_case=True)
train_seq = convert_lines(train_seq, MAX_SEQUENCE_LENGTH, tokenizer)
print('tokenizing complete in {:.0f} seconds.'.format(time.time()-t0))

tokenizing...
tokenizing complete in 3148 seconds.
CPU times: user 52min 36s, sys: 9.05 s, total: 52min 45s
Wall time: 52min 48s


In [15]:
# training preparation

x_train, x_val = train_seq[trn_idx], train_seq[val_idx]
y_train, y_val = train_tars[trn_idx], train_tars[val_idx]
train_loader = prepare_loader(x_train, y_train, batch_size, split='train')
val_loader, val_original_indices = prepare_loader(x_val, y_val, 32, split='valid')
val_df = trn_df.iloc[val_idx]

In [16]:
import gc
del train_seq, train_tars, trn_df
gc.collect()

0

In [17]:
print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
print(str(torch.cuda.memory_cached(device)/1000000 ) + 'M')

0.0M
0.0M


In [18]:
# ft_lrs = [lr, 2*lr]
# model, optimizer = model_optimizer_init(ft_lrs)
# lrs_logs, loss_log = lr_range_test(train_loader, model.to(device), optimizer, UnbiasLoss().to(device), 
#                                    start_lr=(5e-8,1e-7), end_lr=(5,10))
# lrs_logs = list(zip(*lrs_logs))

In [19]:
# fig = plt.figure(figsize=(8,5))
# ax = fig.add_subplot(1,1,1)
# ax.plot(lrs_logs[-1][10:-3], loss_log[10:-3])
# ax.set_xscale('log')
# ax.set_xlabel('Learning rate')
# ax.set_ylabel('Loss');

In [20]:
# model setup
seed_torch(SEED)
torch.cuda.empty_cache()

num_train_steps = int(epochs_for_sched * len(train_loader) / grad_accumulation_steps)
ft_lrs = [lr, 1.1*lr]

model, optimizer = model_optimizer_init(ft_lrs)
scheduler = OneCycleScheduler(optimizer, num_train_steps, sched_profile='linear', max_lr=ft_lrs,
                              div_factor=40, warmup=warmup_proportion)

model = model.to(device)
criterion = UnbiasLoss().to(device)
model, optimizer = amp.initialize(model, optimizer, opt_level="O1", verbosity=0)
solver = NetSolver(model, criterion, optimizer, scheduler, checkpoint_iter, 'epk_1_'+output_model_file)

Building model and optimizer...


In [21]:
print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
print(str(torch.cuda.memory_cached(device)/1000000 ) + 'M')

439.112704M
494.927872M


In [22]:
n_iter = num_train_steps * grad_accumulation_steps
print('Start training.')
t0 = time.time()
solver.train((train_loader, val_loader), n_iter, val_df, val_original_indices, t0)
print('Training complete in {:.0f} seconds.'.format(time.time()-t0))

Start training.
Iteration 20000:
{"metric": "Loss", "value": 0.1743}
{"metric": "AUC", "value": 0.9641}
{"metric": "Val. Loss", "value": 0.1425}
{"metric": "Val. AUC", "value": 0.9643}
{"metric": "Val. Unbiased AUC", "value": 0.9300}
updating best val loss...
updating best val auc...

Iteration 40000:
{"metric": "Loss", "value": 0.1574}
{"metric": "AUC", "value": 0.9727}
{"metric": "Val. Loss", "value": 0.1404}
{"metric": "Val. AUC", "value": 0.9680}
{"metric": "Val. Unbiased AUC", "value": 0.9332}
updating best val loss...
updating best val auc...

Iteration 60000:
{"metric": "Loss", "value": 0.1516}
{"metric": "AUC", "value": 0.9805}
{"metric": "Val. Loss", "value": 0.1392}
{"metric": "Val. AUC", "value": 0.9709}
{"metric": "Val. Unbiased AUC", "value": 0.9393}
updating best val loss...
updating best val auc...

Iteration 80000:
{"metric": "Loss", "value": 0.1483}
{"metric": "AUC", "value": 0.9876}
{"metric": "Val. Loss", "value": 0.1386}
{"metric": "Val. AUC", "value": 0.9717}
{"met

In [23]:
print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
print(str(torch.cuda.memory_cached(device)/1000000 ) + 'M')

2013.807616M
2246.049792M


In [24]:
torch.cuda.empty_cache()

ckpt_weights = [2**e for e in range(len(solver.val_preds))]
val_preds = np.average(solver.val_preds, weights=ckpt_weights, axis=0)
val_unbias_auc = check_unbias_auc(val_df, val_preds, True)
print('{"metric": "Ckpt Val. Unbiased AUC", "value": %.4f}' % (val_unbias_auc,))

                        subgroup  subgroup_size  subgroup_auc  bpsn_auc  \
2      homosexual_gay_or_lesbian           2202      0.856329  0.895445   
7                          white           5089      0.862554  0.907660   
6                          black           3043      0.865930  0.894480   
5                         muslim           4240      0.890181  0.920484   
4                         jewish           1577      0.910818  0.948449   
0                           male           8969      0.936326  0.956691   
1                         female          10817      0.938560  0.960345   
3                      christian           8015      0.943983  0.970264   
8  psychiatric_or_mental_illness           1010      0.945645  0.943163   

   bnsp_auc  
2  0.964740  
7  0.962437  
6  0.967032  
5  0.963635  
4  0.954041  
0  0.961253  
1  0.959621  
3  0.950671  
8  0.973332  
{"metric": "Ckpt Val. Unbiased AUC", "value": 0.9414}
